<a href="https://colab.research.google.com/github/kembanggulaa/Sentiment-Analysis-Twitter/blob/cobaan/Ambil_data_dan_Labeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tweepy
import csv
import pandas as pd
import re

from textblob import TextBlob
!pip install googletrans==4.0.0rc1

import googletrans
from googletrans import *

In [ ]:
#Input API key from twitter
api_key = ""
api_secret_key = ""
access_token = ""
access_token_secret = ""

In [ ]:
#Authentication dari twitter
auth = tweepy.OAuthHandler(api_key, api_secret_key)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [ ]:
#Menasukan kata kunci dan tweet limit
keywords = f'Ibukota baru -filter:retweets'
limit=1000

tweets = tweepy.Cursor(api.search, q=keywords, lang="id", tweet_mode='extended').items(limit)

In [ ]:
#Membuat dataframe
columns = ['User', 'Tweet']
data = []

for tweet in tweets:
  data.append([tweet.user.screen_name, tweet.full_text])

df = pd.DataFrame(data, columns=columns)

print(df)

                User                                              Tweet
0      mirandaagisha  @TirtoID + upah PPH 21 jg ditiadakan, maklum y...
1        spektakuler  Februari 2011, pertama kali kerja di kantor yg...
2    KangGal00341800  Ibukota negara baru didorong pemerataan ekonom...
3             Ancix9  Undang-undang ibukota negara baru resmi disahk...
4             Ancix9  Ibukota negara baru demi transformasi pembangu...
..               ...                                                ...
727         Rizky_WN  @Saham_fess bagus, kalo proyek ibukota baru mu...
728  SimanjuntakElly  @Suara_Bawah Ga boleh pake misal NKRI bubar ba...
729          Resof85  @papa_loren Alasan yg benar itu mau lari dari ...
730      Brata_brati  @geloraco Partai Selingkuh Ibukota.... urus du...
731         toleeeee  @irfannurmhd Lagi gencar cari dana buat ibukot...

[732 rows x 2 columns]


In [ ]:
#Pembersihan data
df["Tweet"] = df["Tweet"].apply(lambda x: ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x).split()))
df["Tweet"] = df["Tweet"].str.replace("[^a-zA-Z#]", " ")
df["Tweet"] = df["Tweet"].str.replace('\\n', '')
df["Tweet"] = df["Tweet"].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
df["Tweet"] = df["Tweet"].str.casefold()
df

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  after removing the cwd from sys.path.


,User,Tweet
0,mirandaagisha,upah ditiadakan maklum pemerintah butuh duit b...
1,spektakuler,februari pertama kali kerja kantor berlokasi j...
2,KangGal00341800,ibukota negara baru didorong pemerataan ekonom...
3,Ancix9,undang undang ibukota negara baru resmi disahk...
4,Ancix9,ibukota negara baru demi transformasi pembangu...
...,...,...
727,Rizky_WN,fess bagus kalo proyek ibukota baru mulai jalan
728,SimanjuntakElly,bawah boleh pake misal nkri bubar bang ibukota...
729,Resof85,loren alasan benar lari dari kenyataan alias k...
730,Brata_brati,partai selingkuh ibukota urus dulu kader parta...


In [ ]:
#Menghapus Tweet yang sama
df.drop_duplicates(inplace = True)
df = df.dropna(axis=0)
df.reset_index(inplace=True)
df.drop('index', axis=1, inplace=True)
df

,User,Tweet
0,mirandaagisha,upah ditiadakan maklum pemerintah butuh duit b...
1,spektakuler,februari pertama kali kerja kantor berlokasi j...
2,KangGal00341800,ibukota negara baru didorong pemerataan ekonom...
3,Ancix9,undang undang ibukota negara baru resmi disahk...
4,Ancix9,ibukota negara baru demi transformasi pembangu...
...,...,...
712,Rizky_WN,fess bagus kalo proyek ibukota baru mulai jalan
713,SimanjuntakElly,bawah boleh pake misal nkri bubar bang ibukota...
714,Resof85,loren alasan benar lari dari kenyataan alias k...
715,Brata_brati,partai selingkuh ibukota urus dulu kader parta...


In [ ]:
#Translate tweet ke bahasa inggris untuk pelabelan
translator = googletrans.Translator()

In [ ]:
df["Tweet"] = df["Tweet"].astype(str)
df["Translate"] = df["Tweet"].apply(translator.translate, src="id", dest="en").apply(getattr, args=("text",))

In [ ]:
#Labeling
df['Sentiment'] = df['Translate'].map(lambda tweet: TextBlob(tweet).sentiment.polarity)

In [ ]:
#Mendefinisikan hasil labeling dengan nilai Netral/Positif/Negatif
def convert(polarity):
  if polarity == 0:
    return "Netral"
  elif polarity > 0:
    return "Positif"
  else:
    return "Negatif"

In [ ]:
df['Hasil'] = df['Sentiment'].apply(convert)
df

,User,Tweet,Translate,Sentiment,Hasil
0,mirandaagisha,upah ditiadakan maklum pemerintah butuh duit b...,wages are negated by the government need a lot...,0.136364,Positif
1,spektakuler,februari pertama kali kerja kantor berlokasi j...,February the first time the office works locat...,0.187273,Positif
2,KangGal00341800,ibukota negara baru didorong pemerataan ekonom...,The new capital city is encouraged by the Nati...,0.187273,Positif
3,Ancix9,undang undang ibukota negara baru resmi disahk...,the new capital law of the new country was off...,0.136364,Positif
4,Ancix9,ibukota negara baru demi transformasi pembangu...,The capital of the new country for the sake of...,0.136364,Positif
...,...,...,...,...,...
712,Rizky_WN,fess bagus kalo proyek ibukota baru mulai jalan,Fess is good if the new capital project starts...,0.418182,Positif
713,SimanjuntakElly,bawah boleh pake misal nkri bubar bang ibukota...,the bottom can use for example the NKRI disban...,0.136364,Positif
714,Resof85,loren alasan benar lari dari kenyataan alias k...,Loren the right reason for running away from r...,0.105519,Positif
715,Brata_brati,partai selingkuh ibukota urus dulu kader parta...,The Cheating Party of the Capital Urus first o...,-0.250000,Negatif


In [ ]:
#Menyimpan data hasil labeling dalam bentuk CSV
df.to_csv("TweetLabeling.csv")